# P-03 Analytics - STR_001_ORB

## Strategy Performance Analytics & Evaluation

This notebook provides comprehensive analytics for STR_001_ORB strategy performance.

In [ ]:
# Import required modules
import sys
from pathlib import Path

# Add project to path
project_root = Path().absolute()
sys.path.insert(0, str(project_root))

from itos.core.strategy_engine import StrategyEngine
from itos.core.analytics_engine import AnalyticsEngine
from itos.core.data_engine import DataEngine

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

print("✅ Modules imported successfully")

## Load Strategy Results

In [ ]:
# Initialize engines
strategy_engine = StrategyEngine()
analytics_engine = AnalyticsEngine()

# Strategy name
strategy_name = "STR_001_ORB"

print(f"📊 Loading results for {strategy_name}...")

# Load strategy results
trades_df = strategy_engine.get_strategy_results(strategy_name)

if trades_df is not None and not trades_df.empty:
    print(f"✅ Loaded {len(trades_df)} trades")
    print(f"   Date Range: {trades_df['entry_time'].min()} to {trades_df['entry_time'].max()}")
    print(f"   Symbols: {trades_df['symbol'].nunique()}")
    
    # Convert datetime columns
    trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
    trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])
    
    # Display sample trades
    display(trades_df.head())
else:
    print(f"❌ No results found for {strategy_name}")
    print("Please run the strategy first using STR_001_strategy_engine.ipynb")

## Basic Performance Metrics

In [ ]:
if trades_df is not None and not trades_df.empty:
    # Basic metrics
    total_trades = len(trades_df)
    winning_trades = len(trades_df[trades_df['pnl'] > 0])
    losing_trades = len(trades_df[trades_df['pnl'] < 0])
    total_pnl = trades_df['pnl'].sum()
    avg_pnl = trades_df['pnl'].mean()
    win_rate = winning_trades / total_trades * 100
    
    # Profit factor
    gross_profit = trades_df[trades_df['pnl'] > 0]['pnl'].sum()
    gross_loss = abs(trades_df[trades_df['pnl'] < 0]['pnl'].sum())
    profit_factor = gross_profit / gross_loss if gross_loss > 0 else float('inf')
    
    # Average win/loss
    avg_win = trades_df[trades_df['pnl'] > 0]['pnl'].mean()
    avg_loss = trades_df[trades_df['pnl'] < 0]['pnl'].mean()
    
    # Largest win/loss
    largest_win = trades_df['pnl'].max()
    largest_loss = trades_df['pnl'].min()
    
    print("📊 Basic Performance Metrics:")
    print(f"   Total Trades: {total_trades}")
    print(f"   Winning Trades: {winning_trades} ({win_rate:.1f}%)")
    print(f"   Losing Trades: {losing_trades} ({100-win_rate:.1f}%)")
    print(f"   Total PnL: ₹{total_pnl:,.2f}")
    print(f"   Average PnL: ₹{avg_pnl:,.2f}")
    print(f"   Profit Factor: {profit_factor:.2f}")
    print(f"   Average Win: ₹{avg_win:,.2f}")
    print(f"   Average Loss: ₹{avg_loss:,.2f}")
    print(f"   Largest Win: ₹{largest_win:,.2f}")
    print(f"   Largest Loss: ₹{largest_loss:,.2f}")
    print(f"   Win/Loss Ratio: {abs(avg_win/avg_loss):.2f}")
else:
    print("❌ No data available")

## Risk Analysis

In [ ]:
if trades_df is not None and not trades_df.empty:
    # Calculate cumulative PnL and drawdown
    trades_df_sorted = trades_df.sort_values('entry_time')
    trades_df_sorted['cumulative_pnl'] = trades_df_sorted['pnl'].cumsum()
    trades_df_sorted['running_max'] = trades_df_sorted['cumulative_pnl'].expanding().max()
    trades_df_sorted['drawdown'] = trades_df_sorted['cumulative_pnl'] - trades_df_sorted['running_max']
    
    # Risk metrics
    max_drawdown = trades_df_sorted['drawdown'].min()
    max_drawdown_pct = (max_drawdown / trades_df_sorted['running_max'].max()) * 100
    
    # Value at Risk
    var_95 = trades_df['pnl'].quantile(0.05)
    var_99 = trades_df['pnl'].quantile(0.01)
    
    # Consecutive losses
    trades_df_sorted['is_loss'] = trades_df_sorted['pnl'] < 0
    trades_df_sorted['loss_group'] = (trades_df_sorted['is_loss'] != trades_df_sorted['is_loss'].shift()).cumsum()
    max_consecutive_losses = trades_df_sorted.groupby('loss_group')['is_loss'].sum().max()
    
    # Sharpe ratio (simplified)
    daily_returns = trades_df.groupby(trades_df['entry_time'].dt.date)['pnl'].sum()
    if len(daily_returns) > 1:
        sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252) if daily_returns.std() > 0 else 0
    else:
        sharpe_ratio = 0
    
    print("⚠️ Risk Analysis:")
    print(f"   Max Drawdown: ₹{max_drawdown:,.2f} ({max_drawdown_pct:.2f}%)")
    print(f"   VaR (95%): ₹{var_95:,.2f}")
    print(f"   VaR (99%): ₹{var_99:,.2f}")
    print(f"   Max Consecutive Losses: {max_consecutive_losses}")
    print(f"   Sharpe Ratio: {sharpe_ratio:.2f}")
    
    # Plot drawdown
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Equity curve
    ax1.plot(trades_df_sorted['entry_time'], trades_df_sorted['cumulative_pnl'], 'b-', label='Equity')
    ax1.set_title(f'{strategy_name} - Equity Curve')
    ax1.set_ylabel('Cumulative PnL')
    ax1.grid(True)
    ax1.legend()
    
    # Drawdown
    ax2.fill_between(trades_df_sorted['entry_time'], trades_df_sorted['drawdown'], 0, 
                     color='red', alpha=0.3)
    ax2.plot(trades_df_sorted['entry_time'], trades_df_sorted['drawdown'], 'r-', label='Drawdown')
    ax2.set_title('Drawdown')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Drawdown')
    ax2.grid(True)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available")

## Time-Based Analysis

In [ ]:
if trades_df is not None and not trades_df.empty:
    # Monthly performance
    trades_df['entry_month'] = trades_df['entry_time'].dt.to_period('M')
    monthly_stats = trades_df.groupby('entry_month').agg({
        'pnl': ['count', 'sum', 'mean'],
        'symbol': 'nunique'
    }).round(2)
    
    monthly_stats.columns = ['Trades', 'Total PnL', 'Avg PnL', 'Symbols']
    
    print("📅 Monthly Performance:")
    display(monthly_stats)
    
    # Day of week analysis
    trades_df['entry_day'] = trades_df['entry_time'].dt.day_name()
    day_stats = trades_df.groupby('entry_day').agg({
        'pnl': ['count', 'sum', 'mean'],
    }).round(2)
    
    day_stats.columns = ['Trades', 'Total PnL', 'Avg PnL']
    day_stats = day_stats.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
    
    print("\n📊 Day of Week Performance:")
    display(day_stats)
    
    # Entry time analysis
    trades_df['entry_hour'] = trades_df['entry_time'].dt.hour
    hour_stats = trades_df.groupby('entry_hour').agg({
        'pnl': ['count', 'sum', 'mean'],
    }).round(2)
    
    hour_stats.columns = ['Trades', 'Total PnL', 'Avg PnL']
    
    print("\n⏰ Entry Hour Performance:")
    display(hour_stats)
    
    # Plot monthly performance
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Monthly PnL
    monthly_pnl = trades_df.groupby('entry_month')['pnl'].sum()
    axes[0, 0].bar(range(len(monthly_pnl)), monthly_pnl.values)
    axes[0, 0].set_title('Monthly PnL')
    axes[0, 0].set_xticks(range(len(monthly_pnl)))
    axes[0, 0].set_xticklabels([str(m) for m in monthly_pnl.index], rotation=45)
    
    # Day of week PnL
    day_pnl = trades_df.groupby('entry_day')['pnl'].sum()
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    day_pnl = day_pnl.reindex(day_order)
    axes[0, 1].bar(range(len(day_pnl)), day_pnl.values)
    axes[0, 1].set_title('Day of Week PnL')
    axes[0, 1].set_xticks(range(len(day_pnl)))
    axes[0, 1].set_xticklabels(day_pnl.index, rotation=45)
    
    # Hourly PnL
    hour_pnl = trades_df.groupby('entry_hour')['pnl'].sum()
    axes[1, 0].bar(hour_pnl.index, hour_pnl.values)
    axes[1, 0].set_title('Hourly PnL')
    axes[1, 0].set_xlabel('Hour')
    
    # Trade count by month
    monthly_count = trades_df.groupby('entry_month')['pnl'].count()
    axes[1, 1].bar(range(len(monthly_count)), monthly_count.values)
    axes[1, 1].set_title('Monthly Trade Count')
    axes[1, 1].set_xticks(range(len(monthly_count)))
    axes[1, 1].set_xticklabels([str(m) for m in monthly_count.index], rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available")

## Symbol Performance Analysis

In [ ]:
if trades_df is not None and not trades_df.empty:
    # Symbol performance
    symbol_stats = trades_df.groupby('symbol').agg({
        'pnl': ['count', 'sum', 'mean', 'std'],
        'entry_time': ['min', 'max']
    }).round(2)
    
    symbol_stats.columns = ['Trades', 'Total PnL', 'Avg PnL', 'Std Dev', 'First Trade', 'Last Trade']
    symbol_stats = symbol_stats.sort_values('Total PnL', ascending=False)
    
    print("📈 Symbol Performance (Top 15):")
    display(symbol_stats.head(15))
    
    # Symbol win rates
    symbol_win_rates = trades_df.groupby('symbol').apply(
        lambda x: len(x[x['pnl'] > 0]) / len(x)
    ).sort_values(ascending=False)
    
    print("\n🎯 Symbol Win Rates (Top 15):")
    display(symbol_win_rates.head(15))
    
    # Bottom performers
    print("\n⚠️ Worst Performing Symbols (Bottom 10):")
    display(symbol_stats.tail(10))
    
    # Plot symbol performance
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Top 10 symbols by PnL
    top_symbols = symbol_stats.head(10)
    axes[0, 0].barh(range(len(top_symbols)), top_symbols['Total PnL'])
    axes[0, 0].set_title('Top 10 Symbols by PnL')
    axes[0, 0].set_yticks(range(len(top_symbols)))
    axes[0, 0].set_yticklabels(top_symbols.index)
    axes[0, 0].set_xlabel('Total PnL')
    
    # Symbol distribution
    symbol_counts = trades_df['symbol'].value_counts().head(20)
    axes[0, 1].bar(range(len(symbol_counts)), symbol_counts.values)
    axes[0, 1].set_title('Trade Count by Symbol (Top 20)')
    axes[0, 1].set_xticks(range(len(symbol_counts)))
    axes[0, 1].set_xticklabels(symbol_counts.index, rotation=45)
    axes[0, 1].set_ylabel('Trade Count')
    
    # PnL distribution
    axes[1, 0].hist(trades_df['pnl'], bins=50, alpha=0.7)
    axes[1, 0].set_title('PnL Distribution')
    axes[1, 0].set_xlabel('PnL')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].axvline(trades_df['pnl'].mean(), color='red', linestyle='--', label='Mean')
    axes[1, 0].legend()
    
    # Win rate distribution
    symbol_win_rate_list = trades_df.groupby('symbol').apply(
        lambda x: len(x[x['pnl'] > 0]) / len(x)
    )
    axes[1, 1].hist(symbol_win_rate_list, bins=20, alpha=0.7)
    axes[1, 1].set_title('Symbol Win Rate Distribution')
    axes[1, 1].set_xlabel('Win Rate')
    axes[1, 1].set_ylabel('Number of Symbols')
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available")

## Exit Reason Analysis

In [ ]:
if trades_df is not None and not trades_df.empty:
    # Exit reason statistics
    exit_stats = trades_df.groupby('exit_reason').agg({
        'pnl': ['count', 'sum', 'mean', 'std'],
        'symbol': 'nunique'
    }).round(2)
    
    exit_stats.columns = ['Count', 'Total PnL', 'Avg PnL', 'Std Dev', 'Symbols']
    
    print("🚪 Exit Reason Analysis:")
    display(exit_stats)
    
    # Win rates by exit reason
    exit_win_rates = trades_df.groupby('exit_reason').apply(
        lambda x: len(x[x['pnl'] > 0]) / len(x)
    )
    
    print("\n🎯 Win Rates by Exit Reason:")
    for exit_reason, win_rate in exit_win_rates.items():
        print(f"   {exit_reason}: {win_rate*100:.1f}%")
    
    # Plot exit reason analysis
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Trade count by exit reason
    exit_counts = trades_df['exit_reason'].value_counts()
    axes[0, 0].bar(exit_counts.index, exit_counts.values)
    axes[0, 0].set_title('Trade Count by Exit Reason')
    axes[0, 0].set_ylabel('Trade Count')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # PnL by exit reason
    exit_pnl = trades_df.groupby('exit_reason')['pnl'].sum()
    axes[0, 1].bar(exit_pnl.index, exit_pnl.values)
    axes[0, 1].set_title('Total PnL by Exit Reason')
    axes[0, 1].set_ylabel('Total PnL')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Win rate by exit reason
    axes[1, 0].bar(exit_win_rates.index, exit_win_rates.values * 100)
    axes[1, 0].set_title('Win Rate by Exit Reason')
    axes[1, 0].set_ylabel('Win Rate (%)')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # PnL distribution by exit reason
    for i, exit_reason in enumerate(exit_counts.index):
        if i < 4:  # Limit to 4 exit reasons
            data = trades_df[trades_df['exit_reason'] == exit_reason]['pnl']
            row = i // 2
            col = i % 2
            axes[1, 1].clear()
            axes[1, 1].hist(data, bins=20, alpha=0.7, label=exit_reason)
            axes[1, 1].set_title(f'PnL Distribution - {exit_reason}')
            axes[1, 1].set_xlabel('PnL')
            axes[1, 1].set_ylabel('Frequency')
            axes[1, 1].legend()
            break
    
    plt.tight_layout()
    plt.show()
else:
    print("❌ No data available")

## Strategy Evaluation & Recommendation

In [ ]:
if trades_df is not None and not trades_df.empty:
    # Run comprehensive analytics
    print("🔍 Running comprehensive analytics...")
    result = analytics_engine.analyze_strategy(strategy_name, trades_df)
    
    # Print comprehensive results
    print("\n" + "="*60)
    print(f"📊 {strategy_name} - Comprehensive Analytics")
    print("="*60)
    
    print(f"\n📋 Performance Summary:")
    print(f"   Total Trades: {result.total_trades}")
    print(f"   Winning Trades: {result.winning_trades}")
    print(f"   Losing Trades: {result.losing_trades}")
    print(f"   Total PnL: ₹{result.total_pnl:,.2f}")
    print(f"   Max Drawdown: {result.max_drawdown*100:.2f}%")
    print(f"   Profit Factor: {result.profit_factor:.2f}")
    print(f"   Sharpe Ratio: {result.sharpe_ratio:.2f}")
    print(f"   Avg Duration: {result.avg_trade_duration:.1f} minutes")
    
    # Get recommendation
    recommendation = analytics_engine.evaluate_strategy(result)
    
    print(f"\n🎯 Strategy Evaluation:")
    print(f"   Recommendation: {recommendation}")
    
    if recommendation == "APPROVE":
        print("   ✅ Strategy APPROVED for live trading")
        print("   📈 Strategy meets all performance criteria")
        print("   🎯 Ready for Phase 3 - Strategy Library")
    elif recommendation == "MODIFY":
        print("   ⚠️ Strategy needs MODIFICATION")
        print("   🔧 Some parameters need adjustment")
        print("   📊 Consider optimizing entry/exit rules")
    else:
        print("   ❌ Strategy KILLED - not viable")
        print("   🚫 Strategy does not meet minimum requirements")
        print("   💀 Discard or completely redesign")
    
    # Evaluation criteria
    print(f"\n📏 Evaluation Criteria:")
    print(f"   Minimum Trades: {result.total_trades} >= 50 {'✅' if result.total_trades >= 50 else '❌'}")
    print(f"   Max Drawdown: {abs(result.max_drawdown)*100:.2f}% <= 20% {'✅' if abs(result.max_drawdown) <= 0.20 else '❌'}")
    print(f"   Profit Factor: {result.profit_factor:.2f} >= 1.2 {'✅' if result.profit_factor >= 1.2 else '❌'}")
    print(f"   Total PnL: ₹{result.total_pnl:,.2f} > 0 {'✅' if result.total_pnl > 0 else '❌'}")
    print(f"   Sharpe Ratio: {result.sharpe_ratio:.2f} >= 0.5 {'✅' if result.sharpe_ratio >= 0.5 else '❌'}")
    
    # Additional insights
    if result.metadata:
        print(f"\n💡 Additional Insights:")
        
        if 'win_rate_analysis' in result.metadata:
            win_data = result.metadata['win_rate_analysis']
            print(f"   Overall Win Rate: {win_data['overall_win_rate']*100:.1f}%")
            print(f"   Symbol Avg Win Rate: {win_data['symbol_avg_win_rate']*100:.1f}%")
        
        if 'risk_metrics' in result.metadata:
            risk_data = result.metadata['risk_metrics']
            print(f"   Value at Risk (95%): ₹{risk_data.get('var_95', 0):,.2f}")
            print(f"   Max Consecutive Losses: {risk_data.get('max_consecutive_losses', 0)}")
            print(f"   Avg Win/Loss Ratio: {risk_data.get('avg_win_to_loss_ratio', 0):.2f}")
    
    print("\n" + "="*60)
else:
    print("❌ No data available for evaluation")

## Summary & Next Steps

In [ ]:
if trades_df is not None and not trades_df.empty:
    print("\n" + "="*60)
    print(f"📊 {strategy_name} Analytics Summary")
    print("="*60)
    
    # Key takeaways
    print(f"\n🎯 Key Takeaways:")
    
    if result.total_pnl > 0:
        print(f"   ✅ Strategy is profitable with ₹{result.total_pnl:,.2f} total PnL")
    else:
        print(f"   ❌ Strategy is unprofitable with ₹{result.total_pnl:,.2f} total PnL")
    
    if result.profit_factor >= 1.2:
        print(f"   ✅ Strong profit factor of {result.profit_factor:.2f}")
    else:
        print(f"   ⚠️ Weak profit factor of {result.profit_factor:.2f}")
    
    if abs(result.max_drawdown) <= 0.20:
        print(f"   ✅ Acceptable drawdown of {abs(result.max_drawdown)*100:.2f}%")
    else:
        print(f"   ❌ High drawdown of {abs(result.max_drawdown)*100:.2f}%")
    
    # Best performing symbols
    best_symbols = symbol_stats.head(3)
    print(f"\n🏆 Best Performing Symbols:")
    for symbol in best_symbols.index:
        pnl = best_symbols.loc[symbol, 'Total PnL']
        trades = best_symbols.loc[symbol, 'Trades']
        win_rate = symbol_win_rates[symbol] * 100
        print(f"   {symbol}: ₹{pnl:,.2f} ({trades} trades, {win_rate:.1f}% win rate)")
    
    # Most common exit reasons
    print(f"\n🚪 Exit Reason Analysis:")
    for exit_reason in exit_counts.index:
        count = exit_counts[exit_reason]
        pnl = exit_pnl[exit_reason]
        win_rate = exit_win_rates[exit_reason] * 100
        print(f"   {exit_reason}: {count} trades, ₹{pnl:,.2f}, {win_rate:.1f}% win rate")
    
    # Recommendations for improvement
    print(f"\n💡 Recommendations for Improvement:")
    
    if result.profit_factor < 1.2:
        print(f"   🔧 Improve profit factor by adjusting stop loss/target levels")
    
    if abs(result.max_drawdown) > 0.15:
        print(f"   ⚠️ Reduce drawdown by implementing tighter risk management")
    
    if result.sharpe_ratio < 0.5:
        print(f"   📈 Improve risk-adjusted returns by filtering low-probability setups")
    
    # Next steps
    print(f"\n📋 Next Steps:")
    
    if recommendation == "APPROVE":
        print(f"   1. ✅ Strategy approved - add to Phase 3 Strategy Library")
        print(f"   2. 📊 Monitor performance in different market conditions")
        print(f"   3. 🔄 Test with out-of-sample data")
        print(f"   4. 🚀 Prepare for live trading deployment")
    elif recommendation == "MODIFY":
        print(f"   1. 🔧 Modify strategy parameters based on analytics")
        print(f"   2. 📊 Re-test with modified parameters")
        print(f"   3. 🎯 Focus on improving weak areas identified")
        print(f"   4. 🔄 Repeat analytics cycle")
    else:
        print(f"   1. ❌ Strategy killed - do not proceed to live trading")
        print(f"   2. 💀 Analyze reasons for failure")
        print(f"   3. 📝 Document lessons learned")
        print(f"   4. 🔄 Develop completely new strategy approach")
    
    print(f"\n📁 Generated Files:")
    print(f"   📊 Analytics Summary: itos/analytics/{strategy_name}_summary.csv")
    print(f"   📈 Dashboard: itos/analytics/{strategy_name}_dashboard.png")
    print(f"   📋 Detailed Analytics: itos/analytics/{strategy_name}_analytics.json")
    
    print("\n" + "="*60)
else:
    print("❌ No analytics data available")